[![Open in Google Earth Engine](https://img.shields.io/badge/Open%20in-Google%20Earth%20Engine-4285F4?style=for-the-badge&logo=google-earth&logoColor=white)](https://code.earthengine.google.com/87ac51fc81a194c7a1dfa299f3251a95)

---
title: "View from outer space"
---

[![Open in Google Earth Engine](https://img.shields.io/badge/Open%20in-Google%20Earth%20Engine-4285F4?style=for-the-badge&logo=google-earth&logoColor=white)](https://code.earthengine.google.com/87ac51fc81a194c7a1dfa299f3251a95)

<iframe height="600" width="100%" frameborder="no" src="https://carlos-mendez.projects.earthengine.app/view/rc-dmsp-ntl?height=600">

</iframe>


-   [Open in full view](https://carlos-mendez.projects.earthengine.app/view/rc-dmsp-ntl)

-   [Run source code in Google Earth Engine](https://code.earthengine.google.com/87ac51fc81a194c7a1dfa299f3251a95?hideCode=true)


```         
/*******************************************
 * GEE Nighttime Lights Visualization Code
 *******************************************/

// WEB APP URL:  https://carlos-mendez.projects.earthengine.app/view/rc-dmsp-ntl

// WEBSITE WITH SOURCE CODE: https://code.earthengine.google.com/87ac51fc81a194c7a1dfa299f3251a95?hideCode=true


/*******************************************
 * Section: Data Loading and Preprocessing
 *******************************************/

// 🛰️ Function to load calibrated nighttime light (NTL) images for a given year
function loadNTL(year) {
  return ee.Image('projects/ee-carlosmendez777/assets/NTL_RC_calibrated/NTL' + year)
    .selfMask(); // Mask out invalid pixels
}

// 🗺️ Load Indian states dataset from geoBoundaries and filter for India
var indianStates = ee.FeatureCollection("projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1")
  .filter(ee.Filter.eq('shapeGroup', 'IND'));

// 💡 Load nighttime lights images for 1996 and 2010
var ntl1996 = loadNTL(1996);
var ntl2010 = loadNTL(2010);

/*******************************************
 * Section: Visualization Parameters
 *******************************************/

// 🎨 Visualization parameters for nighttime lights
var vis = {
  min: 2.0,
  max: 16.0,
  palette: ['#253494', '#2c7fb8', '#41b6c4', '#a1dab4', '#ffffcc'],
  opacity: 0.75  // Opacity defined as 0.75 (i.e., 25% transparency)
};

// 🗺️ Dark Basemap style configuration for UI maps
var styledMapType = {
  Dark: [
    // 🖌️ Text labels styling
    {
      "featureType": "all",
      "elementType": "labels.text.fill",
      "stylers": [
        { "saturation": 36 },
        { "color": "#000000" },
        { "lightness": 40 }
      ]
    },
    // 🖌️ Text stroke styling
    {
      "featureType": "all",
      "elementType": "labels.text.stroke",
      "stylers": [
        { "visibility": "on" },
        { "color": "#000000" },
        { "lightness": 16 }
      ]
    },
    // 🚫 Hide icons
    {
      "featureType": "all",
      "elementType": "labels.icon",
      "stylers": [
        { "visibility": "off" }
      ]
    },
    // 🌍 Administrative area fill
    {
      "featureType": "administrative",
      "elementType": "geometry.fill",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 20 }
      ]
    },
    // 🌍 Administrative area stroke
    {
      "featureType": "administrative",
      "elementType": "geometry.stroke",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 17 },
        { "weight": 1.2 }
      ]
    },
    // 🏞️ Landscape styling
    {
      "featureType": "landscape",
      "elementType": "geometry",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 20 }
      ]
    },
    // 🏙️ Points of Interest styling
    {
      "featureType": "poi",
      "elementType": "geometry",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 21 }
      ]
    },
    // 🚗 Road styling: highways
    {
      "featureType": "road.highway",
      "elementType": "geometry.fill",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 17 }
      ]
    },
    {
      "featureType": "road.highway",
      "elementType": "geometry.stroke",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 29 },
        { "weight": 0.2 }
      ]
    },
    // 🛣️ Road styling: arterial roads
    {
      "featureType": "road.arterial",
      "elementType": "geometry",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 18 }
      ]
    },
    // 🛣️ Road styling: local roads
    {
      "featureType": "road.local",
      "elementType": "geometry",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 16 }
      ]
    },
    // 🚆 Transit styling
    {
      "featureType": "transit",
      "elementType": "geometry",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 19 }
      ]
    },
    // 💧 Water styling
    {
      "featureType": "water",
      "elementType": "geometry",
      "stylers": [
        { "color": "#000000" },
        { "lightness": 17 }
      ]
    }
  ]
};

/*******************************************
 * Section: Map Initialization and UI Setup
 *******************************************/

// 📍 Center coordinates focused on India
var center = { lon: 78.96288, lat: 20.593684, zoom: 6 };

// 🗺️ Create two maps for side-by-side comparison
var leftMap = ui.Map(center);
var rightMap = ui.Map(center);

// 🎛️ Set map options to use Dark Basemap style
leftMap.setOptions('Dark', {'Dark': styledMapType.Dark});
rightMap.setOptions('Dark', {'Dark': styledMapType.Dark});

// 🔗 Link the two maps for synchronized navigation
var linker = ui.Map.Linker([leftMap, rightMap]);

// 📐 Create a split panel to display the maps side-by-side
var splitPanel = ui.SplitPanel({
  firstPanel: leftMap,
  secondPanel: rightMap,
  orientation: 'horizontal',
  wipe: true
});

// 🧹 Clear default map and add the custom split panel to the UI root
ui.root.clear();
ui.root.add(splitPanel);

/*******************************************
 * Section: Layer Addition
 *******************************************/

// 📡 Add nighttime lights layers clipped to Indian states boundaries
leftMap.addLayer(ntl1996.clip(indianStates), vis, 'NTL 1996');
rightMap.addLayer(ntl2010.clip(indianStates), vis, 'NTL 2010');

// 🗺️ Define style for state boundaries and add to both maps
var stateStyle = { color: 'white', fillColor: '00000000', width: 1 };
leftMap.addLayer(indianStates.style(stateStyle), {}, 'State Boundaries');
rightMap.addLayer(indianStates.style(stateStyle), {}, 'State Boundaries');

// 🔖 Add descriptive labels to the maps
var label1 = ui.Label({
  value: 'Luminosity in 1996',
  style: {
    fontSize: '25px',
    backgroundColor: '#f7f7f7',
    position: 'top-left'
  }
});
var label2 = ui.Label({
  value: 'Luminosity in 2010',
  style: {
    fontSize: '25px',
    backgroundColor: '#f7f7f7',
    position: 'top-right'
  }
});
leftMap.add(label1);
rightMap.add(label2);

/*******************************************
 * Section: Legend Creation
 *******************************************/

// 📊 Function to create a legend panel for nighttime light intensity
function createLegendPanel() {
  // 🖼️ Legend panel styling
  var legendPanel = ui.Panel({
    style: {
      position: 'bottom-right',
      padding: '8px',
      backgroundColor: '#f7f7f7',  // Light gray background
      border: '1px solid #cccccc'  // Border for definition
    }
  });

  // 🏷️ Legend title
  var legendTitle = ui.Label({
    value: 'Nighttime Light Intensity',
    style: {
      fontWeight: 'bold',
      fontSize: '16px',
      margin: '0 0 6px 0',
      padding: '0'
    }
  });
  legendPanel.add(legendTitle);

  // 🌈 Create a color bar thumbnail representing the palette
  var colorBar = ui.Thumbnail({
    image: ee.Image.pixelLonLat().select(0),
    params: {
      bbox: [0, 0, 1, 0.1],
      dimensions: '200x10',
      format: 'png',
      min: 0,
      max: 1,
      palette: ['#253494', '#2c7fb8', '#41b6c4', '#a1dab4', '#ffffcc']
    },
    style: {
      stretch: 'horizontal',
      margin: '0 0 6px 0',
      maxHeight: '24px'
    }
  });
  legendPanel.add(colorBar);

  // 🔢 Create a panel for numerical labels corresponding to the color bar
  var labelPanel = ui.Panel({
    widgets: [
      ui.Label(vis.min.toFixed(1), { margin: '0', fontSize: '12px' }),
      ui.Label(
        ((vis.max - vis.min) / 2 + vis.min).toFixed(1),
        { margin: '0', textAlign: 'center', stretch: 'horizontal', fontSize: '12px' }
      ),
      ui.Label(vis.max.toFixed(1), { margin: '0', fontSize: '12px' })
    ],
    layout: ui.Panel.Layout.flow('horizontal'),
    style: { stretch: 'horizontal' }
  });
  legendPanel.add(labelPanel);

  // 💡 Add label for digital number (DN)
  legendPanel.add(ui.Label('Digital number (DN)', {
    margin: '3px 0 0 0',
    fontSize: '12px',
    textAlign: 'center'
  }));

  return legendPanel;
}

// 📊 Create legend panels for each map and add them to the respective maps
var leftLegend = createLegendPanel();
var rightLegend = createLegendPanel();
leftMap.add(leftLegend);
rightMap.add(rightLegend);
```